In [3]:
from ROOT import TFile, TH1D, TH2D, THStack, TCanvas, TLegend, gStyle
from maxpatch_utils import first_year_dataset, three_year_dataset
import pandas as pd
import numpy as np
import pickle
import yaml
import glob
import json
import os

gStyle.SetOptStat(0)

Welcome to JupyROOT 6.24/02


 	Column	Unnamed: 0	BottomRingCharge	ClippedV1720Count	ClippedV1740Count	Fprompt	LL_ar40PDF_MBL_qPE	LL_ar40PDF_TF2_qPE	LL_ar40_2Min_MBL_qPE	LL_ar40_2Min_TF2_qPE	LL_ar40_Min_MBL_qPE	LL_ar40_Min_TF2_qPE	LL_ar40_all_MBL_qPE	LL_ar40_all_TF2_qPE	LL_pmtsPDF_PMTs_qPE	LL_pmtsPDF_gaps_qPE	LL_surfPDF_MBL_qPE	LL_surfPDF_TF2_qPE	MC_KE	MaxFpromptFrac	MaxFpromptShift	NLL_tag_MBL_qPE	NLL_tag_TF2_qPE	NPE	NeckVetoN	PMTID0	PMTID1	PMTID2	PMTID3	PMTID4	PMTID5	PulseIndexFirstGAr	Strip1condition	Strip2condition	Strip3condition	T0	TopRingCharge	TruthR	TruthRho	TruthX	TruthY	TruthZ	distance1	distance2	distance3	distance4	distance5	dmsa	eventID	flashfraction	flashllratio	fmaxnsc	fmaxpe	fraction1	fraction2	fraction3	fraction4	fraction5	gof_ar40PDF_MBL_qPE	gof_ar40PDF_TF2_qPE	gof_pmtsPDF_PMTs_qPE	gof_pmtsPDF_gaps_qPE	gof_surfPDF_MBL_qPE	gof_surfPDF_TF2_qPE	llneck4	llneutronflash	mblikelihoodKS	mblikelihoodKuiper	mblikelihoodR	mblikelihoodRho	mblikelihoodX	mblikelihoodY	mblikelihoodZ	nSCBayes	nhit	npromptrms	nt7030	nt9010	pmt_dof	promptrms	pulseindexfirstgar	qPE	rCutConsistancy	roitop05	roitop30	rprompt60Bayes	runID	sample_name	scintillationlikelihood	scintillationlikelihood16ns	slchargedof	slchargegof	sldof	sldof16ns	slfprompt	slqpe	t7030	t9010	timefit2R	timefit2Rho	timefit2X	timefit2Y	timefit2Z	trigger_count	zCutConsistancy
    

In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
mpl.rcParams['agg.path.chunksize'] = 10000
mpl.rcParams.update({'font.size': 14})
plt.rcParams["figure.titleweight"] = "bold"
plt.rcParams["figure.titlesize"] = 14
mpl.rcParams['figure.facecolor'] = 'white'

In [8]:
### for DCC cuts
usecols = ['sample_name', 'runID', 'eventID','nSCBayes', 'rprompt60Bayes', 'mblikelihoodR', 'mblikelihoodZ', 'fmaxpe',
'zCutConsistancy', 'rCutConsistancy', 'NeckVetoN', 'TopRingCharge', 'BottomRingCharge', 'PulseIndexFirstGAr', 'roitop05', 'roitop30',
'NLL_tag_TF2_qPE', 'NLL_tag_MBL_qPE',
'LL_ar40_Min_MBL_qPE', 'LL_ar40_Min_TF2_qPE',
'distance1', 'fraction1',
'distance2', 'fraction2',
'distance3', 'fraction3',
'distance4', 'fraction4',
'distance5', 'fraction5'
# 'LL_ar40PDF_TF2_qPE_700', 'LL_ar40PDF_TF2_qPE_750'
]

roiContF = TFile("savePLRroi.root","READ")
roiCut30 = roiContF.Get("roi_top30_gr")
roiCut05 = roiContF.Get("roi_top05_gr")
roiCutPlr = roiContF.Get("roi_plr_gr")
roiContF.Close()

roiCut30.SetLineColor(1)
roiCut05.SetLineColor(2)
roiCutPlr.SetLineColor(3)
Canvas = TCanvas ("Canvas", "Canvas", 800, 600);  
#roiCutPlr.Draw()
#Canvas.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas


In [1]:
nll_cut_val = 2 ## sina: for STR cuts initial cut flow
def cutflow(sample_data, MBLR):
	sample_data = sample_data[sample_data['mblikelihoodR']<MBLR]
	sample_data = sample_data[sample_data['mblikelihoodZ']<550]

	sample_data = sample_data[sample_data['fmaxpe']<0.4]
	sample_data = sample_data[sample_data['NeckVetoN']==0]

	sample_data = sample_data[sample_data['TopRingCharge']<0.04]
	sample_data = sample_data[sample_data['BottomRingCharge']<0.1]
	# sample_data = sample_data[sample_data['PulseIndexFirstGAr']>2]

	# sample_data = sample_data[sample_data['zCutConsistancy']==1]
	sample_data = sample_data[sample_data['rCutConsistancy']==1]

	# sample_data = sample_data[sample_data['nSCBayes']<200]
	sample_data = sample_data[sample_data['roitop05']==1 ]
	# sample_data = sample_data[sample_data['Strip2condition']==1]
	sample_data = get_double_cluster_cut(sample_data) ## Sina: cut flow contains DCC!
	
	# sample_data = sample_data[sample_data['rprompt60Bayes']<0.75 ]
	# sample_data = sample_data[sample_data['roitop30']==1]

	return sample_data

In [2]:
def get_double_cluster_cut(sample_data, remove=True):
	if len(sample_data)==0:
		return sample_data
	cut1 = (sample_data.fmaxpe>0.07) & (sample_data.distance1>600) & (sample_data.fraction1>30)
	cut2 = (sample_data.fmaxpe>0.07) & (sample_data.distance2>600) & (sample_data.fraction2>30)
	cut3 = (sample_data.fmaxpe>0.07) & (sample_data.distance3>600) & (sample_data.fraction3>30)
	cut4 = (sample_data.fmaxpe>0.07) & (sample_data.distance4>600) & (sample_data.fraction4>30)
	cut5 = (sample_data.fmaxpe>0.07) & (sample_data.distance5>600) & (sample_data.fraction5>30)
	double_cut = cut1+cut2+cut3+cut4+cut5
	# if remove is set to false, it will return the events that are removed by this cut
	if remove:
		double_cut = [not item for item in double_cut]
	sample_data2 = sample_data[double_cut]
	# print(len(sample_data), len(sample_data2))

	return sample_data2